### Background modeling

### Frame difference method

Since the target in the scene is moving, the position of the target image in different image frames is different. This type of algorithm performs a differential operation on two consecutive frames of images in time. The pixels corresponding to different frames are subtracted to determine the absolute value of the grayscale difference. When the absolute value exceeds a certain threshold, it can be determined to be a moving target, thereby achieving the goal. detection function.
The frame difference method is very simple, but it will introduce noise and hole problems

![title](bg_2.png)

### Gaussian mixture model

Before foreground detection, the background is trained first, and a Gaussian mixture model is used to simulate each background in the image. The number of Gaussian mixtures for each background can be adaptive. Then in the testing phase, GMM matching is performed on the new pixels. If the pixel value can match one of the Gaussians, it is considered to be the background, otherwise it is considered to be the foreground. Since the GMM model is continuously updated and learned throughout the process, it has a certain degree of robustness to dynamic backgrounds. Finally, good results were achieved by performing foreground detection on a dynamic background with swaying branches.

![title](bg_3.png)

The actual distribution of the background should be a mixture of multiple Gaussian distributions, and each Gaussian model can also have weights

![title](bg_4.png)

#### Mixed Gaussian model learning method

- 1. First initialize each Gaussian model matrix parameter.

- 2. Take T frame data images from the video to train the Gaussian mixture model. After getting the first pixel, use it as the first Gaussian distribution.

- 3. When the subsequent pixel value is compared with the mean value of the previous Gaussian, if the value of the pixel is within 3 times the variance of its model mean, it belongs to the distribution, and its parameters are updated.

- 4. If the next coming pixel does not satisfy the current Gaussian distribution, use it to create a new Gaussian distribution.

#### Mixed Gaussian model testing method

In the testing phase, the value of the new pixel is compared with each mean value in the Gaussian mixture model. If the difference is between 2 times the variance, it is considered to be the background, otherwise it is considered to be the foreground. Assign the foreground value to 255 and the background value to 0. This forms a foreground binary image.

![title](bg_5.png)

In [3]:
import numpy as np
import cv2

#测试视频
cap = cv2.VideoCapture('test.avi')
#形态学操作
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
#混合高斯模型用于背景建模
fgbg = cv2.createBackgroundSubtractorMOG2()

while(True):
    ret, frame = cap.read()
    fgmask = fgbg.apply(frame)
    #形态学开运算去噪点
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)
    #寻找视频中的轮廓
    im, contours, hierarchy = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for c in contours:
        #计算各轮廓的周长
        perimeter = cv2.arcLength(c,True)
        if perimeter > 188:
            #找到一个直矩形（不会旋转）
            x,y,w,h = cv2.boundingRect(c)
            #画出矩形
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)    

    cv2.imshow('frame',frame)
    cv2.imshow('fgmask', fgmask)
    k = cv2.waitKey(150) & 0xff
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()
